In [1]:
import sys
sys.path.append('../')

# import os
# os.environ['PYTHONASYNCIODEBUG'] = '1'

# Parallelism and beyond

## Facing the reality of python concurrency, again

## More piping operators

## aiochan without asyncio

In [2]:
import aiochan as ac
import asyncio

async def consumer(c, tag):
    async for v in c:
        print('%s received %s' % (tag, v))
        
async def producer(c, tag):
    for i in range(5):
        v = '%s-%s' % (tag, i)
        print('%s produces %s' % (tag, v))
        await c.put(v)
        
async def main():
    c = ac.Chan()
    for i in range(3):
        ac.go(consumer(c, 'c' + str(i)))
    for i in range(3):
        ac.go(producer(c, 'p' + str(i)))
    await asyncio.sleep(0.1)

ac.run(main())

p0 produces p0-0
p0 produces p0-1
p0 produces p0-2
p0 produces p0-3
p1 produces p1-0
p2 produces p2-0
c0 received p0-0
c0 received p0-3
c0 received p1-0
c0 received p2-0
c1 received p0-1
c2 received p0-2
p0 produces p0-4
p1 produces p1-1
p1 produces p1-2
p1 produces p1-3
p2 produces p2-1
c0 received p0-4
c0 received p1-3
c0 received p2-1
c1 received p1-1
c2 received p1-2
p1 produces p1-4
p2 produces p2-2
p2 produces p2-3
p2 produces p2-4
c0 received p1-4
c0 received p2-4
c1 received p2-2
c2 received p2-3


In [3]:
def consumer(c, tag):
    def cb(v):
        if v is not None:
            print('%s received %s' % (tag, v))
            consumer(c, tag)
    c.get_nowait(immediate_only=False, cb=cb)

def producer(c, tag, i=0):
    v = '%s-%s' % (tag, i)
    def cb(ok):
        if ok and i < 4:
            print('%s produces %s' % (tag, v))
            producer(c, tag, i+1)
    
    c.put_nowait(v, immediate_only=False, cb=cb)
    
def main():
    c = ac.Chan(loop='no_loop')
    for i in range(3):
        consumer(c, 'c' + str(i))
    for i in range(3):
        producer(c, 'p' + str(i))
        
main()

c0 received p0-0
p0 produces p0-0
c1 received p0-1
p0 produces p0-1
c2 received p0-2
p0 produces p0-2
c0 received p0-3
p0 produces p0-3
c1 received p0-4
c2 received p1-0
p1 produces p1-0
c0 received p1-1
p1 produces p1-1
c1 received p1-2
p1 produces p1-2
c2 received p1-3
p1 produces p1-3
c0 received p1-4
c1 received p2-0
p2 produces p2-0
c2 received p2-1
p2 produces p2-1
c0 received p2-2
p2 produces p2-2
c1 received p2-3
p2 produces p2-3
c2 received p2-4


In [4]:
def select_run():
    c = ac.Chan(1, loop='no_loop', name='c')
    d = ac.Chan(1, loop='no_loop', name='d')
    put_chan = None

    def put_cb(v, c):
        nonlocal put_chan
        put_chan = c

    ac.select((c, 1), (d, 2), cb=put_cb)

    get_val = None

    def get_cb(v, c):
        nonlocal get_val
        get_val = v

    ac.select(c, d, cb=get_cb)

    print('select put into %s, get value %s' % (put_chan, get_val))
    
def main():
    for _ in range(10):
        select_run()
        
main()

select put into Chan<c 140329680924288>, get value 1
select put into Chan<c 140329680924288>, get value 1
select put into Chan<c 140329680924288>, get value 1
select put into Chan<d 140329680923768>, get value 2
select put into Chan<d 140329680924288>, get value 2
select put into Chan<c 140329680924288>, get value 1
select put into Chan<d 140329672134936>, get value 2
select put into Chan<c 140329680924288>, get value 1
select put into Chan<c 140329680924288>, get value 1
select put into Chan<d 140329672135664>, get value 2
